In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Import the data set

In [28]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# TODO: Rerun with this cleaning:
# Data cleaning
lab_data = lab_data[lab_data['Thorax_length'] != '.']

In [29]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [30]:
lab_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1730 entries, 0 to 1730
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1730 non-null   object 
 1   Population     1730 non-null   object 
 2   Latitude       1730 non-null   float64
 3   Longitude      1730 non-null   float64
 4   Year_start     1730 non-null   int64  
 5   Year_end       1730 non-null   int64  
 6   Temperature    1730 non-null   int64  
 7   Vial           1730 non-null   int64  
 8   Replicate      1730 non-null   int64  
 9   Sex            1730 non-null   object 
 10  Thorax_length  1730 non-null   float64
 11  l2             1730 non-null   float64
 12  l3p            1730 non-null   float64
 13  l3d            1730 non-null   float64
 14  lpd            1730 non-null   float64
 15  l3             1730 non-null   float64
 16  w1             1730 non-null   float64
 17  w2             1730 non-null   float64
 18  w3           

In [31]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [32]:
#TODO: Currently using decision tree so no need to normalise, but must do for other methods

In [33]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: x['Sex'],
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
923,D._buzzatii,Binjour,-25.52,151.45,1994,1994,25,5,1,female,...,1.847,0.632,1.518,2.149,2.148,0.952,1.313,1.088,1.788,female
927,D._buzzatii,Binjour,-25.52,151.45,1994,1994,25,5,3,female,...,1.887,0.625,1.568,2.193,2.193,0.954,1.348,1.075,1.842,female
1356,D._buzzatii,Grandchester,-27.68,152.45,1994,1994,30,8,1,male,...,1.516,0.502,1.332,1.835,1.835,0.850,1.148,0.933,1.771,male
61,D._aldrichi,Binjour,-25.52,151.45,1994,1994,25,3,1,female,...,0.000,0.600,0.000,0.000,0.000,0.000,1.252,0.000,0.000,female
574,D._aldrichi,Oxford_Downs,-21.77,148.85,1994,1994,25,2,2,female,...,1.807,0.593,1.513,2.106,2.106,0.944,1.289,1.091,1.815,female


In [34]:

train, test = train_test_split(lab_data, test_size=0.3)

In [35]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)
#TODO: Add Thorax Length and wing_loading to this training
classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [36]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [37]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
404,D._aldrichi,Grandchester,-27.68,152.45,1994,1994,25,2,2,female,...,0.599,1.515,2.113,2.112,0.946,1.296,1.072,1.786,female,female
842,D._buzzatii,Binjour,-25.52,151.45,1994,1994,20,1,2,female,...,0.640,1.699,2.339,2.339,1.031,1.436,1.186,1.829,female,female
309,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,30,6,1,female,...,0.522,1.318,1.840,1.838,0.843,1.121,0.940,1.759,female,male
1690,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,30,4,1,female,...,0.540,1.381,1.921,1.920,0.861,1.197,0.938,1.833,female,male
1182,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,30,8,2,male,...,0.491,1.356,1.847,1.846,0.817,1.116,0.946,1.785,male,male


In [38]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

1023

In [39]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

188

In [40]:
# Retry with a deeper depth

In [41]:
deep_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=10)

deep_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [42]:
train['deep_class_predictions'] = deep_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [43]:
#Correct deep predictions
train.apply(
    lambda x: 1 if x['deep_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

1114

In [44]:
min_leaf_classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

min_leaf_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

In [45]:
train['min_leaf_predictions'] = min_leaf_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [46]:
#Correct min_leaf predictions
train.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

1136

In [47]:
fully_fit_classifier = DecisionTreeClassifier(criterion='entropy')

fully_fit_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy')

In [48]:
train['fully_fit_predictions'] = fully_fit_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [49]:
#Correct fully_fit predictions
train.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

1211

In [50]:
# Checking the min leaf and fully fit classifiers against the test set

In [51]:
#Correst min_leaf predictions on the test set

In [52]:
test['min_leaf_predictions'] = min_leaf_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['fully_fit_predictions'] = fully_fit_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])


correct_min_leaf_classification_test = test.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Min leaf correct predictions test set:" + str(correct_min_leaf_classification_test))

correct_fully_fit_classification_test = test.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Fully fit predictions test set:" + str(correct_fully_fit_classification_test))


Min leaf correct predictions test set:421
Fully fit predictions test set:420


In [53]:
print(f"accuracy: {correct_min_leaf_classification_test / len(test)}")

accuracy: 0.8111753371868978
